# Model input ERP format preparation 

In this notebook: 
- Necessary inputs
- Read all epochs
- Function to create dataframe with average mismatch response for all participants (needs to be transformed to function)
- Formatting dataframe as suitable model input

## Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import ipywidgets as widgets
from IPython.display import display

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
#import eegyolk
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata.head()

,eeg_file,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,101a,101,a,m,20,m,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,101a_epo.fif
1,102a,102,a,f,20,Nee,Control,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,102a_epo.fif
2,103a,103,a,f,20,m,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,103a_epo.fif
3,104a,104,a,m,18,f,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,104a_epo.fif
4,105a,105,a,f,17,f,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,105a_epo.fif


## Read all epochs from files

The function below loads all filtered epochs from the dataframe which contains the metadata and the epochs filepaths + filenames. The epochs are arrays for each stimuli with a time interval of -0.2 to 0.8.

In [4]:
epochs = initialization_functions.read_filtered_data(metadata)

Checking out file: 101a_epo.fif
Checking out file: 102a_epo.fif
Checking out file: 103a_epo.fif
Checking out file: 104a_epo.fif
Checking out file: 105a_epo.fif
Checking out file: 106a_epo.fif
Checking out file: 107a_epo.fif
Checking out file: 109a_epo.fif
Checking out file: 110a_epo.fif
Checking out file: 111a_epo.fif
Checking out file: 112a_epo.fif
Checking out file: 114a_epo.fif
Checking out file: 115a_epo.fif
Checking out file: 116a_epo.fif
Checking out file: 117a_epo.fif
Checking out file: 118a_epo.fif
Checking out file: 119a_epo.fif
Checking out file: 124a_epo.fif
Checking out file: 125a_epo.fif
Checking out file: 126a_epo.fif
Checking out file: 127a_epo.fif
Checking out file: 128a_epo.fif
Checking out file: 129a_epo.fif
Checking out file: 130a_epo.fif
Checking out file: 131a_epo.fif
Checking out file: 133a_epo.fif
Checking out file: 135a_epo.fif
Checking out file: 136a_epo.fif
Checking out file: 137a_epo.fif
Checking out file: 138a_epo.fif
Checking out file: 139a_epo.fif
Checking

In [5]:
len(epochs)

43

## Create pandas dataframe with the average difference between standard and deviant responses

The function below needs `metadata`, the loaded `epochs` and the definition of the standard and deviant events as input. You should define your standard and deviant events as an array. In the function `input_mmr_prep` it's important to know that the assumption is made that the deviant follows after a standard event. Therefore the deviant belonging to the standard is the  standard event number + 1. Make sure your events are numbered like this, else the function won't calculate the mismatch response.  

In [6]:
def input_mmr_prep(metadata, epochs, standard_events, deviant_events): 
    # create dataframe with expected columns 
    df = pd.DataFrame(columns=["eeg_file",  "channel", "mean"]) # "paradigm",

    # loop over all participants
    for i in range(len(metadata['eeg_file'])):
        std_evoked = epochs[i][standard_events].average() 
        dev_evoked = epochs[i][deviant_events].average()

        # calculate the mismatch response between standard and deviant evoked
        evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])
        
        # get a list of all channels
        chnames_list = evoked_diff.info['ch_names']
        
        # compute for every channel the features of the mismatch line
        for channel in chnames_list: 
            chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
            roi_dict = dict(left_ROI=chnames) # combine_channels only takes a dictionary as input
            roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
            mmr = roi_evoked.to_data_frame()
            mmr_avg = mmr['left_ROI'].mean()
            mmr_std = mmr['left_ROI'].std()
            mmr_skew = mmr['left_ROI'].skew()
            mmr_var = mmr['left_ROI'].var()
            mmr_kurt = mmr['left_ROI'].kurtosis()
            
            df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew, 'kurt' : mmr_kurt, 'var' : mmr_var}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms 
    return df

In [7]:
# define the events for standard and deviant
standard_events = ['GiepM_S','GiepS_S','GopM_S','GopS_S']
deviant_events = ['GiepM_D','GiepS_D','GopM_D','GopS_D']


df = input_mmr_prep(metadata, epochs, standard_events, deviant_events)

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

...

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-8196e876cdba>:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x0000021D87BECB80>, axis=0)".
  roi_evoke

In [9]:
df

,eeg_file,channel,mean,kurt,skew,std,var
0,101a,Fp1,-1.169016,0.260606,-1.038757,1.613613,2.603748
1,101a,AF3,-0.307508,-0.662227,-0.630391,1.361383,1.853363
2,101a,F7,-1.685437,0.040356,-0.656505,2.439540,5.951354
3,101a,F3,-0.360602,0.142669,-1.027285,1.129224,1.275148
4,101a,FC1,0.037099,-0.704019,0.094718,0.456107,0.208033
...,...,...,...,...,...,...,...
1371,154a,F8,-0.864006,-0.495933,-0.421748,2.275743,5.179008
1372,154a,AF4,-1.661009,-1.246108,-0.297841,2.370142,5.617573
1373,154a,Fp2,-2.313553,-1.406808,-0.293656,2.480706,6.153904
1374,154a,Fz,-1.608168,-1.187018,-0.136414,2.161758,4.673199


In [10]:
df = df.drop_duplicates(subset=['eeg_file','channel']) # ,'paradigm'

## Transpose dataframe into combination of paradigm and channel per participant

We now want a single row for every participant containing the paradigm and corresponding channels. The code below generates this dataframe. 

In [14]:
# transformation of the dataframe
df = df.pivot(index='eeg_file', columns=['channel']) # 'paradigm',

KeyError: "None of ['channel'] are in the columns"

In [12]:
df.columns = ['_'.join(str(s).strip() for s in col if s) for col in df.columns]

,mean_AF3,mean_AF4,mean_C3,mean_C4,mean_CP1,mean_CP2,mean_CP5,mean_CP6,mean_Cz,mean_F3,...,var_Oz,var_P3,var_P4,var_P7,var_P8,var_PO3,var_PO4,var_Pz,var_T7,var_T8
eeg_file,,,,,,,,,,,,,,,,,,,,,
101a,-3.075080e-01,-1.117406e+00,-2.651369e-01,-2.490048,-3.447545e-01,-1.952034,2.442909e-01,-2.060369,-4.303650e-01,-3.606021e-01,...,6.378025,0.736197,4.263168,9.768049e-01,4.081511,2.823812,3.707833,1.865558,1.246092e+00,4.157734
102a,-1.286638e+00,-1.297360e+00,-7.810052e-01,0.558700,3.143731e-01,0.903432,-1.469916e+00,0.213751,8.335533e-01,-1.622451e+00,...,4.422531,0.976890,1.062023,3.117754e+00,4.967815,2.167772,2.121805,0.864177,6.283401e-01,2.540204
103a,-1.565712e+00,-4.819496e-01,1.631278e-01,2.782407,2.052576e+00,3.038309,-2.244892e+00,4.449279,1.505256e+00,-4.685838e-01,...,32.713005,4.177749,6.977577,3.544522e+01,18.145947,5.509024,10.550081,10.769883,4.609330e+00,10.190182
104a,-5.961355e-02,-6.350983e-01,2.397895e-01,0.663436,5.343487e-02,-0.615029,6.137400e-01,1.628879,-9.549421e-03,5.469499e-02,...,3.011483,0.987851,1.963031,3.953997e+00,3.938772,1.845088,2.675353,1.101257,1.635812e+00,6.517099
105a,1.518115e+00,3.254148e+00,4.754453e-01,0.239900,4.446789e-01,-0.945208,1.471047e+00,0.360562,1.157556e-01,2.544221e+00,...,6.542767,1.174915,2.127196,5.014867e+00,5.143383,0.971683,1.688680,1.125295,1.147491e+01,6.965740
106a,1.222853e+00,-2.109778e-01,1.007780e-01,2.893458,-7.918648e-01,1.849126,2.886796e+00,2.436897,3.294207e-01,1.897448e+00,...,2.931006,3.299338,6.747550,7.133543e+00,4.983561,2.879790,4.470761,1.911889,2.732053e+00,2.822238
107a,-4.825081e-01,-7.117283e-01,-5.190678e-01,0.491282,7.493382e-01,0.736852,7.561641e-01,1.853508,8.644503e-02,-3.042896e-01,...,0.662828,1.291536,2.797468,2.482020e+00,4.187095,1.139802,1.415275,0.959948,2.560170e+00,1.665248
109a,-1.186959e+00,-3.300903e-01,-5.284324e-01,-0.087747,-6.632836e-01,-0.062600,-1.667445e+00,-0.688172,-3.753765e-01,-9.308529e-01,...,2.393439,1.283204,0.467251,4.072248e+00,1.737602,1.776235,1.149327,0.759134,3.535508e+00,1.523331
110a,5.291558e-02,1.272940e-01,1.248282e-01,-0.138762,3.944488e-02,0.290254,-2.181961e-01,0.181780,1.967050e+00,-6.563537e-01,...,2.389010,0.431525,0.731973,8.613693e-01,0.927371,0.849840,1.314322,0.703577,1.112128e+00,0.818804


In [13]:
df.reset_index(inplace=True)

,eeg_file,mean_AF3,mean_AF4,mean_C3,mean_C4,mean_CP1,mean_CP2,mean_CP5,mean_CP6,mean_Cz,...,var_Oz,var_P3,var_P4,var_P7,var_P8,var_PO3,var_PO4,var_Pz,var_T7,var_T8
0,101a,-3.075080e-01,-1.117406e+00,-2.651369e-01,-2.490048,-3.447545e-01,-1.952034,2.442909e-01,-2.060369,-4.303650e-01,...,6.378025,0.736197,4.263168,9.768049e-01,4.081511,2.823812,3.707833,1.865558,1.246092e+00,4.157734
1,102a,-1.286638e+00,-1.297360e+00,-7.810052e-01,0.558700,3.143731e-01,0.903432,-1.469916e+00,0.213751,8.335533e-01,...,4.422531,0.976890,1.062023,3.117754e+00,4.967815,2.167772,2.121805,0.864177,6.283401e-01,2.540204
2,103a,-1.565712e+00,-4.819496e-01,1.631278e-01,2.782407,2.052576e+00,3.038309,-2.244892e+00,4.449279,1.505256e+00,...,32.713005,4.177749,6.977577,3.544522e+01,18.145947,5.509024,10.550081,10.769883,4.609330e+00,10.190182
3,104a,-5.961355e-02,-6.350983e-01,2.397895e-01,0.663436,5.343487e-02,-0.615029,6.137400e-01,1.628879,-9.549421e-03,...,3.011483,0.987851,1.963031,3.953997e+00,3.938772,1.845088,2.675353,1.101257,1.635812e+00,6.517099
4,105a,1.518115e+00,3.254148e+00,4.754453e-01,0.239900,4.446789e-01,-0.945208,1.471047e+00,0.360562,1.157556e-01,...,6.542767,1.174915,2.127196,5.014867e+00,5.143383,0.971683,1.688680,1.125295,1.147491e+01,6.965740
5,106a,1.222853e+00,-2.109778e-01,1.007780e-01,2.893458,-7.918648e-01,1.849126,2.886796e+00,2.436897,3.294207e-01,...,2.931006,3.299338,6.747550,7.133543e+00,4.983561,2.879790,4.470761,1.911889,2.732053e+00,2.822238
6,107a,-4.825081e-01,-7.117283e-01,-5.190678e-01,0.491282,7.493382e-01,0.736852,7.561641e-01,1.853508,8.644503e-02,...,0.662828,1.291536,2.797468,2.482020e+00,4.187095,1.139802,1.415275,0.959948,2.560170e+00,1.665248
7,109a,-1.186959e+00,-3.300903e-01,-5.284324e-01,-0.087747,-6.632836e-01,-0.062600,-1.667445e+00,-0.688172,-3.753765e-01,...,2.393439,1.283204,0.467251,4.072248e+00,1.737602,1.776235,1.149327,0.759134,3.535508e+00,1.523331
8,110a,5.291558e-02,1.272940e-01,1.248282e-01,-0.138762,3.944488e-02,0.290254,-2.181961e-01,0.181780,1.967050e+00,...,2.389010,0.431525,0.731973,8.613693e-01,0.927371,0.849840,1.314322,0.703577,1.112128e+00,0.818804
9,111a,-5.810631e-02,-2.655512e-01,-6.713716e-01,-0.468874,3.817968e-01,0.649463,3.666286e-01,0.881873,-2.124109e-01,...,1.147801,0.650845,1.066711,2.613046e+00,4.176062,1.306507,1.022425,2.775691,2.052273e+00,1.760690


## Merge and safe dataframe

We still need to merge some of the metadata into the dataframe, so we have the information of the age, gender and label of the participant. 

In [15]:
df = pd.merge(df, metadata, on='eeg_file')

In [16]:
pd.set_option('display.max_columns', None)

In [17]:
df

,eeg_file,mean_AF3,mean_AF4,mean_C3,mean_C4,mean_CP1,mean_CP2,mean_CP5,mean_CP6,mean_Cz,mean_F3,mean_F4,mean_F7,mean_F8,mean_FC1,mean_FC2,mean_FC5,mean_FC6,mean_Fp1,mean_Fp2,mean_Fz,mean_O1,mean_O2,mean_Oz,mean_P3,mean_P4,mean_P7,mean_P8,mean_PO3,mean_PO4,mean_Pz,mean_T7,mean_T8,kurt_AF3,kurt_AF4,kurt_C3,kurt_C4,kurt_CP1,kurt_CP2,kurt_CP5,kurt_CP6,kurt_Cz,kurt_F3,kurt_F4,kurt_F7,kurt_F8,kurt_FC1,kurt_FC2,kurt_FC5,kurt_FC6,kurt_Fp1,kurt_Fp2,kurt_Fz,kurt_O1,kurt_O2,kurt_Oz,kurt_P3,kurt_P4,kurt_P7,kurt_P8,kurt_PO3,kurt_PO4,kurt_Pz,kurt_T7,kurt_T8,skew_AF3,skew_AF4,skew_C3,skew_C4,skew_CP1,skew_CP2,skew_CP5,skew_CP6,skew_Cz,skew_F3,skew_F4,skew_F7,skew_F8,skew_FC1,skew_FC2,skew_FC5,skew_FC6,skew_Fp1,skew_Fp2,skew_Fz,skew_O1,skew_O2,skew_Oz,skew_P3,skew_P4,skew_P7,skew_P8,skew_PO3,skew_PO4,skew_Pz,skew_T7,skew_T8,std_AF3,std_AF4,std_C3,std_C4,std_CP1,std_CP2,std_CP5,std_CP6,std_Cz,std_F3,std_F4,std_F7,std_F8,std_FC1,std_FC2,std_FC5,std_FC6,std_Fp1,std_Fp2,std_Fz,std_O1,std_O2,std_Oz,std_P3,std_P4,std_P7,std_P8,std_PO3,std_PO4,std_Pz,std_T7,std_T8,var_AF3,var_AF4,var_C3,var_C4,var_CP1,var_CP2,var_CP5,var_CP6,var_Cz,var_F3,var_F4,var_F7,var_F8,var_FC1,var_FC2,var_FC5,var_FC6,var_Fp1,var_Fp2,var_Fz,var_O1,var_O2,var_Oz,var_P3,var_P4,var_P7,var_P8,var_PO3,var_PO4,var_Pz,var_T7,var_T8,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,101a,-3.075080e-01,-1.117406e+00,-2.651369e-01,-2.490048,-3.447545e-01,-1.952034,2.442909e-01,-2.060369,-4.303650e-01,-3.606021e-01,-0.869062,-1.685437e+00,-1.320273,3.709948e-02,-0.283584,-6.776742e-01,-1.250907,-1.169016e+00,-1.749117e+00,-3.305586e-02,-0.597666,-2.596950,-2.514119,-0.644141,-2.824247,1.365969e-02,-1.826525,-1.024139,-2.431471,-1.670712,-1.152493e+00,-1.818161,-0.662227,0.552862,-0.722270,-0.902288,0.417584,0.108099,0.066907,-0.898025,-0.489093,0.142669,-0.200325,0.040356,-0.728821,-0.704019,-0.353652,0.729763,-0.599536,0.260606,0.290342,-0.457298,-1.108639,-1.227771,-1.250857,0.540389,-1.137817,-0.300789,-1.169184,-1.043322,-1.059768,-0.677592,-1.123509,-0.569833,-0.630391,-0.870358,0.076734,0.417313,-0.563143,0.990278,0.392878,-0.310648,0.246699,-1.027285,-0.422932,-0.656505,-0.178348,0.094718,0.455089,-1.048427,-0.336491,-1.038757,-0.790867,-0.375478,-0.213395,-0.205587,-0.440525,-0.434018,0.012159,0.192717,-0.315029,-0.554302,-0.230594,-0.310189,0.221245,-0.596817,1.361383e+00,1.384671e+00,4.809516e-01,1.595303,5.020964e-01,1.123514,7.494154e-01,1.692374,4.109749e-01,1.129224e+00,1.003108,2.439540e+00,1.303306,4.561066e-01,0.501310,1.020202e+00,1.320292,1.613613e+00,1.632540e+00,1.002481e+00,1.934221,2.332280,2.525475,0.858019,2.064744,9.883344e-01,2.020275,1.680420,1.925573,1.365854,1.116285e+00,2.039052,1.853363e+00,1.917314e+00,2.313144e-01,2.544990,2.521008e-01,1.262284,5.616234e-01,2.864129,1.689003e-01,1.275148e+00,1.006226,5.951354e+00,1.698606,2.080332e-01,0.251312,1.040812e+00,1.743170,2.603748e+00,2.665188e+00,1.004969e+00,3.741212,5.439529,6.378025,0.736197,4.263168,9.768049e-01,4.081511,2.823812,3.707833,1.865558,1.246092e+00,4.157734,101,a,m,20,m,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,101a_epo.fif
1,102a,-1.286638e+00,-1.297360e+00,-7.810052e-01,0.558700,3.143731e-01,0.903432,-1.469916e+00,0.213751,8.335533e-01,-1.622451e+00,-0.850951,-8.514519e-01,-0.981683,1.867131e-01,0.133392,-8.249521e-01,0.635551,-3.663723e-01,-1.486409e+00,-1.732565e+00,-0.510521,0.249319,0.222848,1.165104,0.988093,-1.234232e-01,-1.455675,0.825620,0.310969,0.839339,1.769642e-01,-1.267008,-0.810363,-0.931263,-0.856812,0.006851,0.754474,-0.058492,-0.814115,-0.422496,-0.152424,-0.546528,-0.384374,-0.795270,-0.405937,-0.557876,0.885974,-0.266022,-0.541264,-0.961320,-1.342635,-1.078413,-1.045195,-0.601993,-0.857888,1.309573,-1.077642,-0.501454,-1.033690,1.025066,0.209049,-0.437223,-0.381589,-0.326242,-0.117380,-0.1110

Drop some unnecessary columns. 

In [18]:
df = df.drop(['eeg_file',
       'dyslexic_parent', 'path_eeg','path_epoch',
       'epoch_file', 'path_eventmarkers'], axis =1)

In [19]:
df['sex'] = np.where(
    (df['sex']=='m'), 1,0)

df['Group_AccToParents'] = np.where(
    (df['Group_AccToParents']=='At risk'), 1,0)

In [20]:
first = df.pop('Group_AccToParents')
df.insert(0, 'Group_AccToParents', first)

In [21]:
df.to_csv('df_avg_mmr.csv', index=False) # safe dataframe

## PCA analysis on feature reduction 

In [ ]:
X = df.drop('Group_AccToParents',1)
y = df['Group_AccToParents']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
explained_variance = pca.explained_variance_ratio_

In [ ]:
explained_variance

In [ ]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

In [ ]:
X.shape